<a href="https://colab.research.google.com/github/KhueNguyen312/Simple-Chatbot-using-seq2seq-with-attention-model/blob/main/Seq2Seq_chatbot_with_Movie_Corpus_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import os
import nltk
import re
import unicodedata
import pickle
import collections
import itertools
import sys
import time
from tensorflow import keras
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
MODE = 'train'
num_epochs = 200
embedding_size = 256
rnn_size = 512
batch_size = 64
max_len = 100
max_sample = 300000
trained_model_path = '/content/gdrive/My Drive/Colab Notebooks/Trained Models/Seq2Seq_Chatbot/movie_corpus'
file_path_movie_lines = '/content/gdrive/My Drive/Colab Notebooks/Dataset/movie_lines.txt'
file_path_conversations = '/content/gdrive/My Drive/Colab Notebooks/Dataset/movie_conversations.txt'

In [ ]:
class DataPreprocesscing:
  def __init__(self,file_path_movie_lines,file_path_conversations,max_len = 40,max_sample = -1):
    self.path_movie_lines = file_path_movie_lines
    self.path_conversations = file_path_conversations
    self.START = '<start> ' # pad token
    self.END = ' <end>'   # pad token
    self.max_sample = max_sample
    self.max_len = max_len

  #from tensorflow repair dataset
  def load_conversations(self):
    # dictionary of line id to text
    id2line = {}
    with open(self.path_movie_lines, errors='ignore') as file:
      lines = file.readlines()
    for line in lines:
      parts = line.replace('\n', '').split(' +++$+++ ')
      id2line[parts[0]] = parts[4]

    inputs, outputs = [], []
    with open(self.path_conversations, 'r') as file:
      lines = file.readlines()
    for line in lines:
      parts = line.replace('\n', '').split(' +++$+++ ')
      # get conversation in a list of line ID
      conversation = [line[1:-1] for line in parts[3][1:-1].split(', ')]
      for i in range(len(conversation) - 1):
        inputs.append(self.normalize_string(id2line[conversation[i]]))
        outputs.append(self.normalize_string(id2line[conversation[i + 1]]))
        if len(inputs) >= self.max_sample:
          return inputs, outputs
    return inputs, outputs

  def unicode_to_ascii(self,s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
  def normalize_string(self,s):
    s = s.lower().strip()
    s = self.unicode_to_ascii(s)
    s = re.sub(r'([!.?])', r' \1', s)
    s = re.sub(r'[^a-zA-Z.!?]+', r' ', s)
    s = re.sub(r'\s+', r' ', s)
    return s

  def tokenize(self,data):
    data_tokenizer = tf.keras.preprocessing.text.Tokenizer(
        filters='')
    data_tokenizer.fit_on_texts(data)

    tensor = data_tokenizer.texts_to_sequences(data)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                           padding='post')
    return tensor, data_tokenizer

  def process_data(self):
    draw_inputs,draw_outputs = self.load_conversations()
  
    print('\nSample from raw data: ')
    print(draw_inputs[:5])
    print(draw_outputs[:5])

    tokenized_inputs,tokenized_outputs_in,tokenized_outputs_out = [],[],[]
    for (sentence1, sentence2) in zip(draw_inputs, draw_outputs):
      # tokenize sentence
      sentence3 = sentence2 + self.END
      sentence2 = self.START + sentence2
      
      # check tokenized sentence max length
      if len(sentence1) <= self.max_len and len(sentence2) <= self.max_len + 1:
        tokenized_inputs.append(sentence1)
        tokenized_outputs_in.append(sentence2)
        tokenized_outputs_out.append(sentence3)

    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        filters='')
    tokenizer.fit_on_texts(tokenized_inputs)
    tokenizer.fit_on_texts(tokenized_outputs_in)
    tokenizer.fit_on_texts(tokenized_outputs_out)

    inputs = tokenizer.texts_to_sequences(tokenized_inputs)
    dec_in = tokenizer.texts_to_sequences(tokenized_outputs_in)
    dec_out = tokenizer.texts_to_sequences(tokenized_outputs_out)

    inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                           padding='post')
    dec_in = tf.keras.preprocessing.sequence.pad_sequences(dec_in,
                                                           padding='post')
    dec_out = tf.keras.preprocessing.sequence.pad_sequences(dec_out,
                                                           padding='post')

    print('answer input sequences')
    print(dec_in[:2])
    print('answer output sequences')
    print(dec_out[:2])
    return inputs,dec_in,dec_out,tokenizer,tokenized_inputs

In [ ]:
data = DataPreprocesscing(file_path_movie_lines,file_path_conversations,max_len,max_sample)
inputs,dec_in,dec_out,tokenizer,draw_inputs = data.process_data()


Sample from raw data: 
['can we make this quick ? roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad . again .', 'well i thought we d start with pronunciation if that s okay with you .', 'not the hacking and gagging and spitting part . please .', 'you re asking me out . that s so cute . what s your name again ?', 'no no it s my fault we didn t have a proper introduction ']
['well i thought we d start with pronunciation if that s okay with you .', 'not the hacking and gagging and spitting part . please .', 'okay . . . then how bout we try out some french cuisine . saturday ? night ?', 'forget it .', 'cameron .']
answer input sequences
[[    4    29     7  8252    18 11204    18  6653   418     1   144     1
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [    4    86     1     1     1    89    49   692    22   236    57   102
   1094 11915     1  1443     3   15

In [ ]:
draw_inputs[0]

'well i thought we d start with pronunciation if that s okay with you .'

In [ ]:
len(inputs[0])

34

In [ ]:
#create dataset
dataset = tf.data.Dataset.from_tensor_slices(
    (inputs, dec_in, dec_out))
dataset = dataset.shuffle(len(inputs)).batch(
    batch_size, drop_remainder=True)

In [ ]:
# dataset = tf.data.Dataset.from_tensor_slices(
#     (data_en, data_fr_in, data_fr_out))
# dataset = dataset.shuffle(20).batch(batch_size)

Divide the architecture of the model into 3 parts:

-Encoder

-Attention Layers

-Decoder

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self,vocab_size,embedding_size,rnn_size):
    super(Encoder,self).__init__()
    self.rnn_size = rnn_size
    self.embedding = tf.keras.layers.Embedding(vocab_size,embedding_size)
    self.lstm = tf.keras.layers.LSTM(self.rnn_size,return_sequences=True,return_state=True)

  def call(self,input_seq,hidden_state):
    x = self.embedding(input_seq)
    output,state_h,state_c = self.lstm(x,hidden_state)
    return output,state_h,state_c
  def initialize_hidden_state(self,batch_size):
    return (tf.zeros([batch_size,self.rnn_size]),tf.zeros([batch_size,self.rnn_size]))


In [ ]:
vocab_size = len(tokenizer.word_index) + 1
encoder = Encoder(vocab_size, embedding_size, rnn_size)

In [ ]:
init_state = encoder.initialize_hidden_state(1)

output,_,_ = encoder(tf.constant([[1,2,3]]),init_state)
output.shape

TensorShape([1, 3, 512])

In [ ]:
class LuongAttention(tf.keras.Model):
  def __init__(self,rnn_size):
    super(LuongAttention,self).__init__()
    self.wa = tf.keras.layers.Dense(rnn_size)
    
  def call(self,decoder_output,encoder_output):
    #Choose general attention score function
    #decoder_output: (batch_size,1,rnn_size)
    #encoder_output: (batch_size,max_len,rnn_size)
    #score: (batch_size,1,max_len)
    score = tf.matmul(decoder_output,self.wa(encoder_output),transpose_b=True)
    alignment = tf.nn.softmax(score,axis=2)

    #context vector c_t is the average sum of encoder output
    context = tf.matmul(alignment,encoder_output)
    return context, alignment

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self,rnn_size,embedding_size,vocab_size):
    super(Decoder,self).__init__()
    self.attention = LuongAttention(rnn_size)
    self.rnn_size = rnn_size
    self.embedding = tf.keras.layers.Embedding(vocab_size,embedding_size)
    self.lstm = tf.keras.layers.LSTM(rnn_size,return_sequences=True,return_state=True)
    
    self.wc = tf.keras.layers.Dense(rnn_size,activation='tanh')
    self.ws = tf.keras.layers.Dense(vocab_size)
  def call(self,decoder_input,encoder_output,state):
    #decoder_input's shape: (batch_size,1)
    x = self.embedding(decoder_input)
    #lstm_out's shape: (batch_size,1,rnn_size)
    lstm_out,state_h,state_c = self.lstm(x,state)

    #context vector's shape: (batch_size, 1, rnn_size)
    #alignment vector's shape: (batch_size, 1, max_len)
    context,alignment = self.attention(lstm_out,encoder_output)
    lstm_out = tf.concat([tf.squeeze(context,1),tf.squeeze(lstm_out,1)],1)
    lstm_out = self.wc(lstm_out)
    logits = self.ws(lstm_out)
    return logits,state_h,state_c,alignment


In [ ]:
decoder = Decoder(rnn_size, embedding_size,vocab_size )

In [ ]:
initial_state = encoder.initialize_hidden_state(1)
encoder_outputs = encoder(tf.constant([[2.1]]), initial_state)
decoder_outputs = decoder(tf.constant(
    [[2.1]]),encoder_outputs[0] ,encoder_outputs[1:])
decoder_outputs[-1]

<tf.Tensor: shape=(1, 1, 1), dtype=float32, numpy=array([[[1.]]], dtype=float32)>

In [ ]:
def loss_function(targets, logits):
    crossentropy = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True)
    mask = tf.math.logical_not(tf.math.equal(targets, 0))
    mask = tf.cast(mask, dtype=tf.int64)
    loss = crossentropy(targets, logits, sample_weight=mask)

    return loss

optimizer = tf.keras.optimizers.Adam(clipnorm=5.0)

In [ ]:
@tf.function
def train_step(input,target_in,target_out,init_hidden_state):
  loss = 0
  with tf.GradientTape() as tape:
    en_output,en_state_h,en_state_c = encoder(input,init_hidden_state)
    de_state_h = en_state_h
    de_state_c = en_state_c
    #Create a loop to iterate through the target sequences
    for i in range(target_out.shape[1]):
      #expand one dim because input must have shape of (batch_size,max_length)
      de_input = tf.expand_dims(target_in[:,i],1)
      logit,de_state_h,de_state_c,_ = decoder(de_input,en_output,(de_state_h,de_state_c))

      loss += loss_function(target_out[:,i],logit)
    
  batch_loss = loss/target_out.shape[1]

  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return batch_loss

In [ ]:
def predict(test_source_text=None):
    if test_source_text is None:
      test_source_text = draw_inputs[np.random.choice(len(draw_inputs))]
    print(test_source_text)
    test_source_seq = tokenizer.texts_to_sequences([test_source_text])
    print(test_source_seq)

    en_initial_states = encoder.initialize_hidden_state(1)
    en_output,de_state_h, de_state_c = encoder(tf.constant(test_source_seq), en_initial_states)

    de_input = tf.constant([[tokenizer.word_index['<start>']]])
    out_words = []
    alignments = []

    while True:
        de_output, de_state_h, de_state_c, alignment = decoder(
            de_input, en_output, (de_state_h, de_state_c))
        de_input = tf.expand_dims(tf.argmax(de_output, -1), 0)
        out_words.append(tokenizer.index_word[de_input.numpy()[0][0]])
        
        alignments.append(alignment.numpy())

        if out_words[-1] == '<end>' or len(out_words) >= max_len:
            break

    print(' '.join(out_words))
    return np.array(alignments), test_source_text.split(' '), out_words

In [ ]:
if MODE == 'train':
    for e in range(32,num_epochs):
        en_initial_states = encoder.initialize_hidden_state(batch_size)
        if e % 5 == 0:
          encoder.save_weights(trained_model_path
              + '/encoder_{}.h5'.format(e + 1))
          decoder.save_weights(trained_model_path + '/decoder_{}.h5'.format(e + 1))
        for batch, (source_seq, target_seq_in, target_seq_out) in enumerate(dataset.take(-1)):
            loss = train_step(source_seq, target_seq_in,target_seq_out, en_initial_states)

            if batch % 100 == 0:
                print('Epoch {} Batch {} Loss {:.4f}'.format(
                    e + 1, batch, loss.numpy()))
        try:
            predict()

            predict("How are you today ?")
        except Exception:
            continue

Epoch 33 Batch 0 Loss 0.8616
Epoch 33 Batch 100 Loss 0.7755
Epoch 33 Batch 200 Loss 0.7409
Epoch 33 Batch 300 Loss 0.7668
Epoch 33 Batch 400 Loss 0.8124
Epoch 33 Batch 500 Loss 0.8205
Epoch 33 Batch 600 Loss 0.9312
Epoch 33 Batch 700 Loss 0.8254
Epoch 33 Batch 800 Loss 1.0402
Epoch 33 Batch 900 Loss 0.8243
Epoch 33 Batch 1000 Loss 0.8470
Epoch 33 Batch 1100 Loss 0.8332
Epoch 33 Batch 1200 Loss 0.9130
Epoch 33 Batch 1300 Loss 0.8066
Epoch 33 Batch 1400 Loss 0.8548
Epoch 33 Batch 1500 Loss 0.8899
Epoch 33 Batch 1600 Loss 0.9844
Epoch 33 Batch 1700 Loss 0.8340
Epoch 33 Batch 1800 Loss 0.7787
Epoch 33 Batch 1900 Loss 0.8952
Epoch 33 Batch 2000 Loss 0.8075
Epoch 33 Batch 2100 Loss 0.8169
Epoch 33 Batch 2200 Loss 0.8229
Epoch 33 Batch 2300 Loss 0.7983
Epoch 33 Batch 2400 Loss 1.0184
Epoch 33 Batch 2500 Loss 0.8987
well thank you . anyway time for bed . or . . . sofa bed .
[[58, 161, 2, 1, 377, 87, 32, 466, 1, 112, 1, 1, 1, 5690, 466, 1]]
umm <end>
How are you today ?
[[49, 37, 2, 305, 3]]
go

In [ ]:
if MODE == 'train':
    for e in range(51,num_epochs):
        en_initial_states = encoder.initialize_hidden_state(batch_size)
        if e % 5 == 0:
          encoder.save_weights(trained_model_path
              + '/encoder_{}.h5'.format(e + 1))
          decoder.save_weights(trained_model_path + '/decoder_{}.h5'.format(e + 1))
        for batch, (source_seq, target_seq_in, target_seq_out) in enumerate(dataset.take(-1)):
            loss = train_step(source_seq, target_seq_in,target_seq_out, en_initial_states)

            if batch % 100 == 0:
                print('Epoch {} Batch {} Loss {:.4f}'.format(
                    e + 1, batch, loss.numpy()))
        try:
            predict()

            predict("How are you today ?")
        except Exception:
            continue

Epoch 52 Batch 0 Loss 0.4185
Epoch 52 Batch 100 Loss 0.4916
Epoch 52 Batch 200 Loss 0.4382
Epoch 52 Batch 300 Loss 0.5023
Epoch 52 Batch 400 Loss 0.5779
Epoch 52 Batch 500 Loss 0.5068
Epoch 52 Batch 600 Loss 0.4444
Epoch 52 Batch 700 Loss 0.4899
Epoch 52 Batch 800 Loss 0.4954
Epoch 52 Batch 900 Loss 0.5489
Epoch 52 Batch 1000 Loss 0.5519
Epoch 52 Batch 1100 Loss 0.5092
Epoch 52 Batch 1200 Loss 0.5491
Epoch 52 Batch 1300 Loss 0.5198
Epoch 52 Batch 1400 Loss 0.6197
Epoch 52 Batch 1500 Loss 0.5524
Epoch 52 Batch 1600 Loss 0.5416
Epoch 52 Batch 1700 Loss 0.5287
Epoch 52 Batch 1800 Loss 0.5627
Epoch 52 Batch 1900 Loss 0.5833
Epoch 52 Batch 2000 Loss 0.5406
Epoch 52 Batch 2100 Loss 0.5865
Epoch 52 Batch 2200 Loss 0.6048
Epoch 52 Batch 2300 Loss 0.5659
Epoch 52 Batch 2400 Loss 0.5503
Epoch 52 Batch 2500 Loss 0.5595
you didn t happen to call the house last night did you grady ?
[[2, 84, 12, 430, 9, 143, 7, 254, 175, 159, 67, 2, 1993, 3]]
no . <end>
How are you today ?
[[49, 37, 2, 305, 3]]
tha

KeyboardInterrupt: ignored

In [ ]:
encoder_checkpoint = tf.train.latest_checkpoint(trained_model_path + '/encoder/encoder_31.h5')
decoder_checkpoint = tf.train.latest_checkpoint(trained_model_path + '/decoder/')
if encoder_checkpoint is not None:
  print(trained_model_path + '/encoder')
encoder.load_weights(trained_model_path + '/encoder_51.h5')
decoder.load_weights(trained_model_path + '/decoder_51.h5')

In [ ]:
encoder.save_weights(trained_model_path
              + '/encoder_{}.h5'.format(53 + 1))
decoder.save_weights(trained_model_path + '/decoder_{}.h5'.format(53 + 1))

In [ ]:
predict('hello')

hello
[[273]]
. . <end>


(array([[[[1.]]],
 
 
        [[[1.]]],
 
 
        [[[1.]]]], dtype=float32), ['hello'], ['.', '.', '<end>'])